In [1]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [2]:
import nltk
import numpy as np
import re
import string

import seaborn as sns
import matplotlib.pyplot as plt
from nltk.chunk import ne_chunk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [3]:
from pandarallel import pandarallel
import spacy

In [4]:
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_finalproject"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_finalproject connected.


In [5]:
df_final5 = pd.read_parquet("gs://nlp_finalproject/FilteredDataset.parquet", engine='pyarrow')
df_final5.head(5)

,url,date,language,title,text,year,cleaned,clean_title,relevant,tokens,no_tokens,percentage
0,http://galusaustralis.com/2020/02/486473/legaltech-artificial-intelligence-market-2019-technology-advancement-and-future-scope-casetext-inc-catalyst-repository-systems-ebrevia/,2020-02-26,en,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis \n\nGalus Australis\n\nBusinessGeneral NewsHealthcareIndustryInternationalLifestyleSci-Tech\n\nWednesday, February 26 2020\n\nTrending\n\nNeedle Counters Market Comprehensive Study by Companies Medline Industries, Boen Healthcare\nSkin Scrub Trays Market Comprehensive Study by Companies Medline Industries, BD, Deroyal\nGlobal P...",2020,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope Casetext Inc., Catalyst Repository Systems, eBREVIA Galus Australis Galus Australis BusinessGeneral NewsHealthcareIndustryInternationalLifestyleSci-Tech Wednesday, February 26 2020 Trending Needle Counters Market Comprehensive Study by Companies Medline Industries, Boen Healthcare Skin Scrub Trays Market Comprehensive Study by Companies Medline Industries, BD, Deroyal Global Portable Handheld ...","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope Casetext Inc., Catalyst Repository Systems, eBREVIA Galus Australis Galus Australis BusinessGeneral NewsHealthcareIndustryInternationalLifestyleSci-Tech Wednesday, February 26 2020 Trending Needle Counters Market Comprehensive Study by Companies Medline Industries, Boen Healthcare Skin Scrub Trays Market Comprehensive Study by Companies Medline Industries, BD, Deroyal Global Portable Handheld ...",1,"[LegalTech, Artificial, Intelligence, Market, 2019, Technology, Advancement, and, Future, Scope, Casetext, Inc., ,, Catalyst, Repository, Systems, ,, eBREVIA, Galus, Australis, Galus, Australis, BusinessGeneral, NewsHealthcareIndustryInternationalLifestyleSci-Tech, Wednesday, ,, February, 26, 2020, Trending, Needle, Counters, Market, Comprehensive, Study, by, Companies, Medline, Industries, ,, Boen, Healthcare, Skin, Scrub, Trays, Market, Comprehensive, Study, by, Companies, Medline, Industr...",1087,2.628812
4,http://www.millenniumpost.in/big-stories/ai-express-aircraft-skids-off-runaway-in-kozhikode-rescue-ops-on-414926,2020-08-07,en,Two dead as AI Express flight skids off Kozhikode airport,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInlandRoutesBeaconInsightGame OnSafariTrendingCandid TalkGastronomyFact FilesMapping the states of IndiaX/sunday-post/beyond-bygone/sunday-post/epic-power/fact-files/ians-feeds/sundaypost/in-retrospect/mapping-the-states-of-india/puja-special/Reminiscence/sponsored/sundaypost/global-eye/feat...,2020,Two dead as AI Express flight skids off Kozhikode airportTop Toggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInlandRoutesBeaconInsightGame OnSafariTrendingCandid TalkGastronomyFact FilesMapping the states of IndiaX/sunday-post/beyond-bygone/sunday-post/epic-power/fact-files/ians-feeds/sundaypost/in-retrospect/mapping-the-states-of-india/puja-special/Reminiscence/ /sundaypost/global-eye/features/nation/world/te...,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInlandRoutesBeaconInsightGame OnSafariTrendingCandid TalkGastronomyFact FilesMapping the states of IndiaX/sunday-post/beyond-bygone/sunday-post/epic-power/fact-files/ia

In [6]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [7]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.4 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Cleaning to find ORG's in News Text

In [8]:
pandarallel.initialize(progress_bar=True, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
df_final5 = df_final5.drop(['relevant','tokens','url','language'],axis=1)

In [10]:
df_final5['NER_clean'] = df_final5['cleaned'].apply(lambda x: re.sub(r'[^\w\s.,]', '', x))

In [11]:
df_final5['NER_clean'] = df_final5['NER_clean'].str.replace('\d+', '', regex=True)

In [12]:
df_final5['NER_clean'] = df_final5['NER_clean'].apply(lambda x: re.sub(r'{So}', '', x))

In [13]:
df_final5['NER_clean'] = df_final5['NER_clean'].str.strip()

In [14]:
df_final5['year'] = df_final5['date'].dt.year

In [15]:
df_final5['year'].value_counts()

year
2023    100465
2022     22879
2021     19093
2020     15189
2024      5695
Name: count, dtype: int64

### NER to Find ORG and Person in News text

In [16]:
nlp = spacy.load("en_core_web_sm")

In [17]:
df_final5 = df_final5.drop(['no_tokens','percentage'],axis=1)

In [18]:
def extract_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return entities

df_final5['ORG_Ent'] = df_final5['NER_clean'].parallel_apply(extract_entities)

In [19]:
%%time
def extract_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ == "Person"]
    return entities

df_final5['Person_Ent'] = df_final5['NER_clean'].parallel_apply(extract_entities)

CPU times: user 2min 41s, sys: 35 s, total: 3min 16s
Wall time: 1h 12min 38s


In [20]:
df_final5 = df_final5.drop('NER_clean',axis=1)

In [27]:
df_final5.head(5)

,date,title,text,year,cleaned,clean_title,ORG_Ent,Person_Ent
0,2020-02-26,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis \n\nGalus Australis\n\nBusinessGeneral NewsHealthcareIndustryInternationalLifestyleSci-Tech\n\nWednesday, February 26 2020\n\nTrending\n\nNeedle Counters Market Comprehensive Study by Companies Medline Industries, Boen Healthcare\nSkin Scrub Trays Market Comprehensive Study by Companies Medline Industries, BD, Deroyal\nGlobal P...",2020,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope Casetext Inc., Catalyst Repository Systems, eBREVIA Galus Australis Galus Australis BusinessGeneral NewsHealthcareIndustryInternationalLifestyleSci-Tech Wednesday, February 26 2020 Trending Needle Counters Market Comprehensive Study by Companies Medline Industries, Boen Healthcare Skin Scrub Trays Market Comprehensive Study by Companies Medline Industries, BD, Deroyal Global Portable Handheld ...","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope Casetext Inc., Catalyst Repository Systems, eBREVIA Galus Australis Galus Australis BusinessGeneral NewsHealthcareIndustryInternationalLifestyleSci-Tech Wednesday, February 26 2020 Trending Needle Counters Market Comprehensive Study by Companies Medline Industries, Boen Healthcare Skin Scrub Trays Market Comprehensive Study by Companies Medline Industries, BD, Deroyal Global Portable Handheld ...","[LegalTech Artificial Intelligence Market Technology Advancement and Future Scope, Casetext Inc., Catalyst Repository Systems, Companies Medline Industries, Boen Healthcare Skin Scrub Trays Market Comprehensive Study, Companies Medline Industries, Sony, Roche Diagnostics Veterinary Dental Xray Generators Market Comprehensive Study, Gowllands Limited, Nasiff Associates Veterinary Test Strips Market Comprehensive Study, Companies Analyticon Biotechnologies, Heska, Jorgensen Laboratories Anima...",[]
4,2020-08-07,Two dead as AI Express flight skids off Kozhikode airport,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInlandRoutesBeaconInsightGame OnSafariTrendingCandid TalkGastronomyFact FilesMapping the states of IndiaX/sunday-post/beyond-bygone/sunday-post/epic-power/fact-files/ians-feeds/sundaypost/in-retrospect/mapping-the-states-of-india/puja-special/Reminiscence/sponsored/sundaypost/global-eye/feat...,2020,Two dead as AI Express flight skids off Kozhikode airportTop Toggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInlandRoutesBeaconInsightGame OnSafariTrendingCandid TalkGastronomyFact FilesMapping the states of IndiaX/sunday-post/beyond-bygone/sunday-post/epic-power/fact-files/ians-feeds/sundaypost/in-retrospect/mapping-the-states-of-india/puja-special/Reminiscence/ /sundaypost/global-eye/features/nation/world/te...,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInlandRoutesBeaconInsightGame OnSafariTrendingCandid TalkGastronomyFact FilesMapping the states of IndiaX/sunday-post/beyond-bygone/sunday-post/epic-power/fact-files/ians-feeds/sundaypost/in-retrospect/mapping-the-states-of-india/puja-special/Reminiscence/sponsored/sundaypost/global-eye/feat...,"[AI Express, PostBeyond BygoneEpic PowerIn RetrospectGlobal, AI Express, Kozhikode, AI Express, Kozhikode airportXIANS, Air India Express, Local Self Government, MoreLatest News]",[]
7,2021-02-25,MulticoreWare In

### Saving the NER Dataframe

In [28]:
df_final5.to_parquet('gs://nlp_finalproject/NER_Clean.parquet')